## World rotation

Here we demonstrate how to use the EZKL package to rotate an on-chain world. 

![zk-gaming-diagram-transformed](https://hackmd.io/_uploads/HkApuQGV6.png)
> **A typical ZK application flow**. For the shape rotators out there — this is an easily digestible example. A user computes a ZK-proof that they have calculated a valid rotation of a world. They submit this proof to a verifier contract which governs an on-chain world, along with a new set of coordinates, and the world rotation updates. Observe that it’s possible for one player to initiate a *global* change.


In [ ]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass



In [ ]:
from torch import nn
import ezkl
import os
import json
import torch
import math

# these are constatns for the rotation
phi = torch.tensor(5 * math.pi / 180)
s = torch.sin(phi)
c = torch.cos(phi)


class RotateStuff(nn.Module):
    def __init__(self):
        super(RotateStuff, self).__init__()

        # create a rotation matrix -- the matrix is constant and is transposed for convenience
        self.rot = torch.stack([torch.stack([c, -s]),
                   torch.stack([s, c])]).t()

    def forward(self, x):
        x_rot = x @ self.rot   # same as x_rot = (rot @ x.t()).t() due to rot in O(n) (SO(n) even)
        return x_rot


circuit = RotateStuff()

This will showcase the principle directions of rotation by plotting the rotation of a single unit vector.

In [ ]:
from matplotlib import pyplot
pyplot.figure(figsize=(3, 3))
pyplot.arrow(0, 0, 1, 0, width=0.02, alpha=0.5)
pyplot.arrow(0, 0, 0, 1, width=0.02, alpha=0.5)
pyplot.arrow(0, 0, circuit.rot[0, 0].item(), circuit.rot[0, 1].item(), width=0.02)
pyplot.arrow(0, 0, circuit.rot[1, 0].item(), circuit.rot[1, 1].item(), width=0.02)


In [ ]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [ ]:


# initial principle vectors for the rotation are as in the plot above
x = torch.tensor([[1, 0], [0, 1]], dtype=torch.float32)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      )

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


### World rotation in 2D on-chain

For demo purposes we deploy these coordinates to a contract running locally using Anvil. This creates our on-chain world. We then rotate the world using the EZKL package and submit the proof to the contract. The contract then updates the world rotation. For demo purposes we do this repeatedly, rotating the world by 1 transform each time.

In [ ]:
import subprocess
import time
import threading

# make sure anvil is running locally
# $ anvil -p 3030

RPC_URL = "http://localhost:3030"

# Save process globally
anvil_process = None

def start_anvil():
    global anvil_process
    if anvil_process is None:
        anvil_process = subprocess.Popen(["anvil", "-p", "3030", "--code-size-limit=41943040"])
        if anvil_process.returncode is not None:
            raise Exception("failed to start anvil process")
        time.sleep(3)

def stop_anvil():
    global anvil_process
    if anvil_process is not None:
        anvil_process.terminate()
        anvil_process = None


We define our `PyRunArgs` objects which contains the visibility parameters for out model. 
- `input_visibility` defines the visibility of the model inputs
- `param_visibility` defines the visibility of the model weights and constants and parameters 
- `output_visibility` defines the visibility of the model outputs

Here we create the following setup:
- `input_visibility`: "public"
- `param_visibility`: "fixed"
- `output_visibility`: public

In [ ]:
py_run_args = ezkl.PyRunArgs()
py_run_args.input_visibility = "public"
py_run_args.output_visibility = "public"
py_run_args.param_visibility = "private" # private by default
py_run_args.scale_rebase_multiplier = 10

res = ezkl.gen_settings(model_path, settings_path, py_run_args=py_run_args)

In [ ]:
res = ezkl.compile_circuit(model_path, compiled_model_path, settings_path)
assert res == True

We also define a contract that holds out test data. This contract will contain in its storage the data that we will read from and attest to. In production you would not need to set up a local anvil instance. Instead you would replace RPC_URL with the actual RPC endpoint of the chain you are deploying your verifiers too, reading from the data on said chain.

In [ ]:
ezkl.setup_test_evm_witness(
    data_path,
    compiled_model_path,
    # we write the call data to the same file as the input data
    data_path,
    input_source=ezkl.PyTestDataSource.OnChain,
    output_source=ezkl.PyTestDataSource.File,
    rpc_url=RPC_URL)

As we use Halo2 with KZG-commitments we need an SRS string from (preferably) a multi-party trusted setup ceremony. For an overview of the procedures for such a ceremony check out [this page](https://blog.ethereum.org/2023/01/16/announcing-kzg-ceremony). The `get_srs` command retrieves a correctly sized SRS given the calibrated settings file from [here](https://github.com/han0110/halo2-kzg-srs). 

These SRS were generated with [this](https://github.com/privacy-scaling-explorations/perpetualpowersoftau) ceremony. 

In [ ]:
# srs path
res = ezkl.get_srs( settings_path)

In [ ]:
# now generate the witness file 

witness = ezkl.gen_witness(data_path, compiled_model_path, witness_path)
assert os.path.isfile(witness_path)

Here we setup verifying and proving keys for the circuit. As the name suggests the proving key is needed for ... proving and the verifying key is needed for ... verifying. 

In [ ]:
res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

We can now create an EVM verifier contract from our circuit. This contract will be deployed to the chain we are using. In this case we are using a local anvil instance.

In [ ]:
abi_path = 'test.abi'
sol_code_path = 'test.sol'

res = ezkl.create_evm_verifier(
        vk_path,
        
        settings_path,
        sol_code_path,
        abi_path,
    )
assert res == True

In [ ]:
import json

addr_path_verifier = "addr_verifier.txt"

res = ezkl.deploy_evm(
    addr_path_verifier,
    sol_code_path,
    'http://127.0.0.1:3030'
)

assert res == True

With the vanilla verifier deployed, we can now create the data attestation contract, which will read in the instances from the calldata to the verifier, attest to them, call the verifier and then return the result. 



In [ ]:
abi_path = 'test.abi'
sol_code_path = 'test.sol'
input_path = 'input.json'

res = ezkl.create_evm_data_attestation(
        vk_path,
        
        settings_path,
        sol_code_path,
        abi_path,
        input_path,
    )

In [ ]:
addr_path_da = "addr_da.txt"

res = ezkl.deploy_da_evm(
        addr_path_da,
        input_path,
        settings_path,
        sol_code_path,
        RPC_URL,
    )

Now we can pull in the data from the contract and calculate a new set of coordinates. We then rotate the world by 1 transform and submit the proof to the contract. The contract could then update the world rotation (logic not inserted here). For demo purposes we do this repeatedly, rotating the world by 1 transform. 

In [ ]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        
        "single",
    )

print(res)
assert os.path.isfile(proof_path)

Call the view only verify method on the contract to verify the proof. Since it is a view function this is safe to use in production since you don't have to pass your private key.

In [ ]:
# read the verifier address
addr_verifier = None
with open(addr_path_verifier, 'r') as f:
    addr = f.read()
#read the data attestation address
addr_da = None
with open(addr_path_da, 'r') as f:
    addr_da = f.read()

res = ezkl.verify_evm(
    proof_path,
    addr,
    RPC_URL,
    addr_da,
)

As a sanity check lets plot the rotations of the unit vectors. We can see that the unit vectors rotate as expected by the output of the circuit. 

In [ ]:
witness['outputs'][0][0]

In [ ]:
settings = json.load(open(settings_path, 'r'))
out_scale = settings["model_output_scales"][0]

from matplotlib import pyplot
pyplot.figure(figsize=(3, 3))
pyplot.arrow(0, 0, 1, 0, width=0.02, alpha=0.5)
pyplot.arrow(0, 0, 0, 1, width=0.02, alpha=0.5)

arrow_x = ezkl.vecu64_to_float(witness['outputs'][0][0], out_scale)
arrow_y = ezkl.vecu64_to_float(witness['outputs'][0][1], out_scale)
pyplot.arrow(0, 0, arrow_x, arrow_y, width=0.02)
arrow_x = ezkl.vecu64_to_float(witness['outputs'][0][2], out_scale)
arrow_y = ezkl.vecu64_to_float(witness['outputs'][0][3], out_scale)
pyplot.arrow(0, 0, arrow_x, arrow_y, width=0.02)